# RSA Attack using Coppersmith Theorem

Step 1: Initialise RSA values

In [1]:
# initialise my RSA
p = 12869795083300688777180977539261293509742299186625597947007346052371885560376413412969538812952531788746738888696190568941522462811532669027187276982231027
q = 12148353615556056154472437834933029851320451773077215044227637191346624898202059936109696194890997673818589947427245092370775886593279153368858540169203579
N = 156346821631681477401124926750495286740792129458553032234494386688081624853288356506937772456803832811784712148005027115830528920515787519711656110036850378116016773959834119229061429110509500800467962987062370248424826070636629116147537042363934287867297000405976659980678829286591117944009130578389173245633
e = 5
print(gcd((p-1)*(q-1),e))

1


Step 2: Generate stereotyped message
According to Coppersmith, if missing only N^1/e of message then considered small root
Using theorem, can find small roots mathematically

In [2]:
# Stereotyped message
length_N = 1024

unknown_m = "rain"

um_ascii = [ord(i) for i in unknown_m]
u_m = ''.join(map(str, um_ascii))
length_m = int(u_m).bit_length()
print(f"Want to find: {u_m}")

# attach "unknown_m" to arbitrary message
K = ZZ(u_m)
Kdigits = K.digits(2)
M = [0]*length_m + [1]*(length_N-length_m);   # generate "unknown_m" zeros and "1024-unknown_m" ones
for i in range(len(Kdigits)):                 # insert "unknown_m" into least sig bits
    M[i] = Kdigits[i]

# generate ciphertext
ZmodN = Zmod(N)
M = ZZ(M, 2)                                  # convert M into integers
c = ZmodN(M)^e
print(f"ciphertext : {c}")

Want to find: 11497105110
ciphertext : 67373992767897182007036525278010881575274144880767368457641721706933093232940017948571693456110907321347258070229564152878046700085563917015213969025219930448796937856773411173744563559548924436659320098498006743228955681325678572098709910283318328470675785764932426632475956154923442024533324742959746135457


In [3]:
# Defining the Equation
R.<x> = PolynomialRing(ZmodN)               # x is part of integer ring mod N
eqn = (2^length_N - 2^length_m + x)^e - c   # equation as defined by coppersmith

In [4]:
# Variables to adjust
beta = 0.9                                   # must satisfy 0 < beta <= 1
deg = eqn.degree()
epsilon = beta / 7                           # as defined by coppersmith theorem
copp_m = ceil(beta**2 / (deg * epsilon))     # computing m as defined by coppersmith for later polynomials
copp_t = floor(deg * copp_m * ((1/beta) - 1))    # computing t as defined by coppersmith for later polynomials
copp_X = ceil(N**((beta**2/deg) - epsilon))  # boundary X, which root must not exceed

In [11]:
# Coppersmith function
def coppersmith_theorem(eqn, deg, N, beta, copp_m, copp_t, copp_X):

    # applying Howgrave-Graham Theorem, we change polynomial ring of x from (int mod N) to just int
    eqn_z = eqn.change_ring(ZZ)
    x = eqn_z.parent().gen()

    # generate the polynomials to be used in lattice
    g = []
    for ii in range(copp_m):                                            # note i and j represent imaginary in sage
        for jj in range(deg):
            g.append(((x*copp_X)^jj)*N^(copp_m-ii)*eqn_z(x*copp_X)^ii) # g polynomial
    for ii in range(copp_t):
        g.append(((x*copp_X)^ii)*eqn_z(x*copp_X)^copp_m)               # h polynomial
    
    # create lattice for LLL
    lat_length = copp_t + (deg*copp_m)
    lati = Matrix(ZZ, lat_length)                     # square matrix with lat_length width
    
    # fill up the lattice
    for ii in range(lat_length):
        for jj in range(ii+1):
            lati[ii, jj] = g[ii][jj]

    # apply the LLL algorithm
    lati = lati.LLL()

    # take out all the shortest poly to test their root
    new_polynomial = 0
    for ii in range(lat_length):
        new_polynomial += (x^ii)*lati[0,ii]/copp_X^ii

    # solve for the root
    short_roots = new_polynomial.roots()

    # test roots
    roots = []
    for root in short_roots:
        if root[0].is_integer():
            result = eqn_z(ZZ(root[0]))
            if gcd(N, result) >= N^beta:
                roots.append(ZZ(root[0]))

    return roots

In [10]:
roots = coppersmith_theorem(eqn, deg, N, beta, copp_m, copp_t, copp_X)

print("we want to find:",str(u_m))
print("we found:", str(roots))

we want to find: 11497105110
we found: [11497105110]
